In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
# note: size = 10, bagging, nval 7
mname = 'kaz0z'

nval = 0
# nval = 7

size = 10
windows = [3]
days_back_confimed = [1,5,10,20,50,100,250,500,1000]
days_back_fatalities = [1,2,5,10,20,50]

size_group = 10
windows_group = [3,5]
days_back_confimed_group = [1,10,100]

seed = 456
nbag = 1
target_average = 1
# ntrain = 47

In [3]:
import math
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from datetime import date, datetime, timedelta


def rmsle(y, y_pred):
        assert len(y) == len(y_pred)
        terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
        return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5
    

def fix_target(frame, key, target, new_target_name="target"):
    import numpy as np

    corrections = 0
    group_keys = frame[ key].values.tolist()
    target = frame[target].values.tolist()

    for i in range(1, len(group_keys) - 1):
        previous_group = group_keys[i - 1]
        current_group = group_keys[i]

        previous_value = target[i - 1]
        current_value = target[i]
        if current_group == previous_group:
                if current_value<previous_value:
                    current_value=previous_value
                    target[i] =current_value


        target[i] =max(0,target[i] )#correct negative values

    frame[new_target_name] = np.array(target)
    
    
def rate(frame, key, target, new_target_name="rate"):
    import numpy as np


    corrections = 0
    group_keys = frame[ key].values.tolist()
    target = frame[target].values.tolist()
    rate=[1.0 for k in range (len(target))]

    for i in range(1, len(group_keys) - 1):
        previous_group = group_keys[i - 1]
        current_group = group_keys[i]

        previous_value = target[i - 1]
        current_value = target[i]
         
        if current_group == previous_group:
            if previous_value!=0.0:
                rate[i] = current_value/previous_value

        rate[i] = max(1,rate[i] )#correct negative values

    frame[new_target_name] = np.array(rate)
    
def get_data_by_key(dataframe, key, key_value, fields=None):
    mini_frame=dataframe[dataframe[key]==key_value]
    if not fields is None:                
        mini_frame=mini_frame[fields].values
        
    return mini_frame

In [4]:
# directory="/kaggle/input/covid19-global-forecasting-week-3/"
# model_directory="/kaggle/input/modelv3-dir/model"
directory="./"
model_directory="./modelv4-dir/"
geo_dir=None#"/kaggle/input/covid19-forecasting-metadata/"
extra_stable_columns=None#["lat","lon","population","area","density"] #"continent"
group_by_columns=None #["Country_Region"]#["continent"]

if not group_by_columns is None and "continent" in group_by_columns:
    assert not geo_dir is None

train = pd.read_csv(directory + "train.csv", parse_dates=["Date"] , engine="python")
test = pd.read_csv(directory + "test.csv", parse_dates=["Date"], engine="python")

# drop last nval days from train
qtrain = train.Date <= train["Date"].max() - timedelta(nval)
train = train[qtrain]

train["key"]=train[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)
test["key"]=test[["Province_State","Country_Region"]].apply(lambda row: str(row[0]) + "_" + str(row[1]),axis=1)
if not geo_dir is None:
    region_metadata=pd.read_csv(geo_dir+"region_metadata.csv")
    train=pd.merge(train,region_metadata, how="left", left_on=["Province_State","Country_Region"], right_on=["Province_State","Country_Region"] )
    test=pd.merge(test,region_metadata, how="left", left_on=["Province_State","Country_Region"], right_on=["Province_State","Country_Region"] )
    train.to_csv(directory + "train_plus_geo.csv", index=False)
    
print(train.shape, test.shape)

(26292, 7) (13459, 5)


In [5]:
group_names=None

if not group_by_columns is None and len(group_by_columns)>0:
    group_names=[]
    for group in group_by_columns:
        groupss=train[["Date", group,target1,target2]]
        grp=groupss.groupby(["Date", group], as_index=False).sum()
        grp.columns=["Date", group,group +"_" + target1,group +"_" + target2 ]
        train=pd.merge(train,grp, how="left", left_on=["Date", group,group], right_on=["Date", group,group] )
        #group_names+=[group +"_" + target1,group +"_" + target2]
        for gr in [group +"_" + target1,group +"_" + target2]:
            rate(train, key, gr, new_target_name="rate_" +gr ) 
            group_names+=["rate_" +gr]
        train.to_csv(directory + "train_plus_groups.csv", index=False)
        
        
    
# last day in train
max_train_date = train["Date"].max()
max_test_date = test["Date"].max()
# horizon = 2
horizon = (max_test_date-max_train_date).days
print ("horizon", int(horizon))
print ("max_train_date", (max_train_date))
print ("max_test_date", (max_test_date))

#test_new=pd.merge(test,train, how="left", left_on=["key","Date"], right_on=["key","Date"] )
#train.to_csv(directory + "transfomed.csv")

target1="ConfirmedCases"
target2="Fatalities"

key="key"

horizon 30
max_train_date 2020-04-14 00:00:00
max_test_date 2020-05-14 00:00:00


In [6]:
max_train_date - timedelta(1)

Timestamp('2020-04-13 00:00:00')

In [7]:
fix_target(train, key, target1, new_target_name=target1)
fix_target(train, key, target2, new_target_name=target2)

rate(train, key, target1, new_target_name="rate_" +target1 )
rate(train, key, target2, new_target_name="rate_" +target2 )
unique_keys=train[key].unique()
print(len(unique_keys))


train

313


Id Province_State Country_Region       Date  ConfirmedCases  \
0          1            NaN    Afghanistan 2020-01-22             0.0   
1          2            NaN    Afghanistan 2020-01-23             0.0   
2          3            NaN    Afghanistan 2020-01-24             0.0   
3          4            NaN    Afghanistan 2020-01-25             0.0   
4          5            NaN    Afghanistan 2020-01-26             0.0   
...      ...            ...            ...        ...             ...   
26287  35648            NaN       Zimbabwe 2020-04-10            13.0   
26288  35649            NaN       Zimbabwe 2020-04-11            14.0   
26289  35650            NaN       Zimbabwe 2020-04-12            14.0   
26290  35651            NaN       Zimbabwe 2020-04-13            17.0   
26291  35652            NaN       Zimbabwe 2020-04-14            17.0   

       Fatalities              key  rate_ConfirmedCases  rate_Fatalities  
0             0.0  nan_Afghanistan             1.000000              1.0  
1             0.0  nan_Afghanistan             1.000000              1.0  
2             0.0  nan_Afghanistan             1.000000              1.0  
3             0.0  nan_Afghanistan             1.000000              1.0  
4             0.0  nan_Afghanistan             1.000000              1.0  
...           ...              ...                  ...              ...  
26287         3.0     nan_Zimbabwe             1.181818              1.0  
26288         3.0     nan_Zimbabwe             1.076923              1.0  
26289         3.0     nan_Zimbabwe             1.000000              1.0  
26290         3.0     nan_Zimbabwe             1.214286              1.0  
26291         3.0     nan_Zimbabwe             1.000000              1.0  

[26292 rows x 9 columns]

In [8]:
def get_lags(rate_array, current_index, size=20):
    lag_confirmed_rate=[-1 for k in range(size)]
    for j in range (0, size):
        if current_index-j>=0:
            lag_confirmed_rate[j]=rate_array[current_index-j]
        else :
            break
    return lag_confirmed_rate

def days_ago_thresold_hit(full_array, indx, thresold):
        days_ago_confirmed_count_10=-1
        if full_array[indx]>thresold: # if currently the count of confirmed is more than 10
            for j in range (indx,-1,-1):
                entered=False
                if full_array[j]<=thresold:
                    days_ago_confirmed_count_10=abs(j-indx)
                    entered=True
                    break
                if entered==False:
                    days_ago_confirmed_count_10=100 #this value would we don;t know it cross 0      
        return days_ago_confirmed_count_10 
    
    
def ewma_vectorized(data, alpha):
    sums=sum([ (alpha**(k+1))*data[k] for  k in range(len(data)) ])
    counts=sum([ (alpha**(k+1)) for  k in range(len(data)) ])
    return sums/counts

def generate_ma_std_window(rate_array, current_index, size=20, window=3):
    ma_rate_confirmed=[-1 for k in range(size)]
    std_rate_confirmed=[-1 for k in range(size)] 
    
    for j in range (0, size):
        if current_index-j>=0:
            ma_rate_confirmed[j]=np.mean(rate_array[max(0,current_index-j-window+1 ):current_index-j+1])
            std_rate_confirmed[j]=np.std(rate_array[max(0,current_index-j-window+1 ):current_index-j+1])           
        else :
            break
    return ma_rate_confirmed, std_rate_confirmed

def generate_ewma_window(rate_array, current_index, size=20, window=3, alpha=0.05):
    ewma_rate_confirmed=[-1 for k in range(size)]

    
    for j in range (0, size):
        if current_index-j>=0:
            ewma_rate_confirmed[j]=ewma_vectorized(rate_array[max(0,current_index-j-window+1 ):current_index-j+1, ], alpha)           
        else :
            break
    
    #print(ewma_rate_confirmed)
    return ewma_rate_confirmed


def get_target(rate_col, indx, horizon=33, average=target_average, use_hard_rule=False):
    target_values = [-1 for k in range(horizon)]
    cou = 0
    for j in range(indx+1, indx+1+horizon):
        if j < len(rate_col):
            if average==1:
                target_values[cou] = rate_col[j]
            else:
                if use_hard_rule and j + average <= len(rate_col) :
                     target_values[cou]=np.mean(rate_col[j:j+average])
                else:
                    target_values[cou] = np.mean(rate_col[j:min(len(rate_col),j+average)])
                   
            cou += 1
        else :
            break
    return target_values


def derive_features(frame, confirmed, fatalities, rate_confirmed, rate_fatalities, 
                    horizon, size=20, windows=[3,7], days_back_confimed=[1,10,100], days_back_fatalities=[1,2,10], 
                    extra_data=None, groups_data=None, windows_group=[3,7], size_group=20,
                    days_back_confimed_group=[1,10,100]):
    targets=[]
    if not extra_data is None:
        assert len(extra_stable_columns)==extra_data.shape[1]
        
    if not groups_data is None:
        assert len(group_names)==groups_data.shape[1]        
        
    names=["lag_confirmed_rate" + str(k+1) for k in range (size)]
    for day in days_back_confimed:
        names+=["days_ago_confirmed_count_" + str(day) ]
    for window in windows:        
        names+=["ma" + str(window) + "_rate_confirmed" + str(k+1) for k in range (size)]
        names+=["std" + str(window) + "_rate_confirmed" + str(k+1) for k in range (size)] 
        names+=["ewma" + str(window) + "_rate_confirmed" + str(k+1) for k in range (size)]         
        
        
    names+=["lag_fatalities_rate" + str(k+1) for k in range (size)]
    for day in days_back_fatalities:
        names+=["days_ago_fatalitiescount_" + str(day) ]    
    for window in windows:        
        names+=["ma" + str(window) + "_rate_fatalities" + str(k+1) for k in range (size)]
        names+=["std" + str(window) + "_rate_fatalities" + str(k+1) for k in range (size)]  
        names+=["ewma" + str(window) + "_rate_fatalities" + str(k+1) for k in range (size)]        
    names+=["confirmed_level"]
    names+=["fatalities_level"]   
    if not extra_data is None: 
        names+=[k for k in extra_stable_columns]
    if not groups_data is None:  
         for gg in range (groups_data.shape[1]):
             names+=["lag_rate_group_"+ str(gg+1) + "_" + str(k+1) for k in range (size_group)]    
             for day in days_back_confimed_group:
                names+=["days_ago_grooupcount_" + str(gg+1) + "_" + str(day) ]             
             for window in windows_group:        
                names+=["ma_group_" + str(gg+1) + "_" + str(window) + "_rate_" + str(k+1) for k in range (size_group)]
                names+=["std_group_" + str(gg+1)+ "_" + str(window) + "_rate_" + str(k+1) for k in range (size_group)]  
                #names+=["ewma_group_" + str(gg+1) + "_" + str(window) + "_rate_" + str(k+1) for k in range (size)]  

            
    names+=["confirmed_plus" + str(k+1) for k in range (horizon)]    
    names+=["fatalities_plus" + str(k+1) for k in range (horizon)]  
    
    #names+=["current_confirmed"]
    #names+=["current_fatalities"]    
    
    features=[]
    for i in range (len(confirmed)):
        row_features=[]
        #####################lag_confirmed_rate       
        lag_confirmed_rate=get_lags(rate_confirmed, i, size=size)
        row_features+=lag_confirmed_rate
        #####################days_ago_confirmed_count_10
        for day in days_back_confimed:
            days_ago_confirmed_count_10=days_ago_thresold_hit(confirmed, i, day)               
            row_features+=[days_ago_confirmed_count_10] 
        #####################ma_rate_confirmed       
        #####################std_rate_confirmed 
        for window in windows:
            ma3_rate_confirmed,std3_rate_confirmed= generate_ma_std_window(rate_confirmed, i, size=size, window=window)
            row_features+= ma3_rate_confirmed   
            row_features+= std3_rate_confirmed          
            ewma3_rate_confirmed=generate_ewma_window(rate_confirmed, i, size=size, window=window, alpha=0.05)
            row_features+= ewma3_rate_confirmed              
        #####################lag_fatalities_rate   
        lag_fatalities_rate=get_lags(rate_fatalities, i, size=size)
        row_features+=lag_fatalities_rate
        #####################days_ago_confirmed_count_10
        for day in days_back_fatalities:
            days_ago_fatalitiescount_2=days_ago_thresold_hit(fatalities, i, day)               
            row_features+=[days_ago_fatalitiescount_2]     
        #####################ma_rate_fatalities       
        #####################std_rate_fatalities 
        for window in windows:        
            ma3_rate_fatalities,std3_rate_fatalities= generate_ma_std_window(rate_fatalities, i, size=size, window=window)
            row_features+= ma3_rate_fatalities   
            row_features+= std3_rate_fatalities  
            ewma3_rate_fatalities=generate_ewma_window(rate_fatalities, i, size=size, window=window, alpha=0.05)
            row_features+= ewma3_rate_fatalities                  
        ##################confirmed_level
        confirmed_level=0
        
        """
        if confirmed[i]>0 and confirmed[i]<1000:
            confirmed_level= confirmed[i]
        else :
            confirmed_level=2000
        """   
        confirmed_level= confirmed[i]
        row_features+=[confirmed_level]
        ##################fatalities_is_level
        fatalities_is_level=0
        """
        if fatalities[i]>0 and fatalities[i]<100:
            fatalities_is_level= fatalities[i]
        else :
            fatalities_is_level=200            
        """
        fatalities_is_level= fatalities[i]
        
        row_features+=[fatalities_is_level] 
        
        if not extra_data is None:    
            row_features+=extra_data[i].tolist()
            
        if not groups_data is None:  
          for gg in range (groups_data.shape[1]): 
             ## lags per group
             this_group=groups_data[:,gg].tolist()
             lag_group_rate=get_lags(this_group, i, size=size_group)
             row_features+=lag_group_rate           
             #####################days_ago_confirmed_count_10
             for day in days_back_confimed_group:
                days_ago_groupcount_2=days_ago_thresold_hit(this_group, i, day)               
                row_features+=[days_ago_groupcount_2]     
             #####################ma_rate_fatalities       
             #####################std_rate_fatalities 
             for window in windows_group:        
                ma3_rate_group,std3_rate_group= generate_ma_std_window(this_group, i, size=size_group, window=window)
                row_features+= ma3_rate_group   
                row_features+= std3_rate_group             
            
            
        ####################### confirmed_plus target
        confirmed_plus = get_target(rate_confirmed, i, horizon=horizon)
        row_features += confirmed_plus  
        
        ####################### fatalities_plus target
        
        fatalities_plus = get_target(rate_fatalities, i, horizon=horizon)
        row_features += fatalities_plus 
        
        ##################current_confirmed
        #row_features+=[confirmed[i]]
        ##################current_fatalities
        #row_features+=[fatalities[i]]        
        
          
        features.append(row_features)
        
    new_frame=pd.DataFrame(data=features, columns=names).reset_index(drop=True)
    frame=frame.reset_index(drop=True)
    frame=pd.concat([frame, new_frame], axis=1)
    #print(frame.shape)
    return frame
    
    
def feature_engineering_for_single_key(frame, group, key, horizon=33, size=20, windows=[3,7], 
                    days_back_confimed=[1,10,100], days_back_fatalities=[1,2,10],
                    extra_stable_=None, group_nams=None,windows_group=[3,7], 
                    size_group=20, days_back_confimed_group=[1,10,100]):
    
    mini_frame=get_data_by_key(frame, group, key, fields=None)
    
    mini_frame_with_features=derive_features(mini_frame, mini_frame["ConfirmedCases"].values,
            mini_frame["Fatalities"].values, mini_frame["rate_ConfirmedCases"].values, 
            mini_frame["rate_Fatalities"].values, horizon ,size=size, windows=windows,
            days_back_confimed=days_back_confimed, days_back_fatalities=days_back_fatalities,
            extra_data=mini_frame[extra_stable_].values if not extra_stable_ is None else None,
            groups_data=mini_frame[group_nams].values if not group_nams is None else None,
            windows_group=windows_group, size_group=size_group, 
            days_back_confimed_group=days_back_confimed_group)
    #print (mini_frame_with_features.shape[0])
    return mini_frame_with_features

In [9]:
from tqdm import tqdm
train_frame=[]

#print (len(train['key'].unique()))
for unique_k in tqdm(unique_keys):
    mini_frame=feature_engineering_for_single_key(train, key, unique_k,
        horizon=horizon, size=size, 
        windows=windows, days_back_confimed=days_back_confimed,
        days_back_fatalities=days_back_fatalities,
        extra_stable_=extra_stable_columns if extra_stable_columns is not None and len(extra_stable_columns)>0 else None,
        group_nams=group_names,windows_group=windows_group, 
        size_group=size_group, days_back_confimed_group=days_back_confimed_group
        ).reset_index(drop=True) 
    #print (mini_frame.shape[0])
    train_frame.append(mini_frame)
    
train_frame = pd.concat(train_frame, axis=0).reset_index(drop=True)
#train_frame.to_csv(directory +"all" + ".csv", index=False)
new_unique_keys = train_frame['key'].unique()
for kee in new_unique_keys:
    if kee not in unique_keys:
        print (kee , " is not there ")

100%|██████████| 313/313 [00:26<00:00, 11.73it/s]


In [10]:
# data scraped from https://www.worldometers.info/coronavirus/, including past daily snapshots
# download html for final day (country and us states) at 22:00 UTC and run wm0d.ipynb first
wm = pd.read_csv('wmc.csv', parse_dates=["Date"])
cpd = ['Country_Region','Province_State','Date']
# 12 new features, all log1p transformed, must be lagged
wmf = [c for c in wm.columns if c not in cpd]
print(wmf, len(wmf))

# since wm leads by a day, shift the date to make it contemporaneous
wmax = wm.Date.max()
dmax = train_frame.Date.max()
woff = (dmax - wmax).days
print(dmax, wmax, woff)
wm1 = wm.copy()
# wm1['Date'] = wm1.Date + timedelta(woff)
wm1['Date'] = wm1.Date + timedelta(-1)

wm1.Date.value_counts()[:10]

['ActiveCases', 'Deaths1M', 'NewCases', 'NewDeaths', 'SeriousCritical', 'Tests1M', 'TotCases1M', 'TotDeaths1M', 'TotalCases', 'TotalDeaths', 'TotalRecovered', 'TotalTests'] 12
2020-04-14 00:00:00 2020-04-15 00:00:00 -1


2020-04-11    318
2020-04-10    318
2020-04-12    318
2020-04-13    318
2020-04-09    317
2020-04-08    316
2020-04-05    315
2020-04-06    315
2020-04-07    315
2020-04-04    314
Name: Date, dtype: int64

In [11]:
train_frame = train_frame.merge(wm1, how='left', on=cpd)
print(train_frame.shape)

(26292, 178)


In [12]:
train_frame['TotalCases'].describe()

count    12587.000000
mean         3.942764
std          2.754017
min          0.000000
25%          1.609438
50%          3.433987
75%          6.016157
max         12.272703
Name: TotalCases, dtype: float64

In [13]:
train_frame.head()

Id Province_State Country_Region       Date  ConfirmedCases  Fatalities  \
0   1            NaN    Afghanistan 2020-01-22             0.0         0.0   
1   2            NaN    Afghanistan 2020-01-23             0.0         0.0   
2   3            NaN    Afghanistan 2020-01-24             0.0         0.0   
3   4            NaN    Afghanistan 2020-01-25             0.0         0.0   
4   5            NaN    Afghanistan 2020-01-26             0.0         0.0   

               key  rate_ConfirmedCases  rate_Fatalities  lag_confirmed_rate1  \
0  nan_Afghanistan                  1.0              1.0                  1.0   
1  nan_Afghanistan                  1.0              1.0                  1.0   
2  nan_Afghanistan                  1.0              1.0                  1.0   
3  nan_Afghanistan                  1.0              1.0                  1.0   
4  nan_Afghanistan                  1.0              1.0                  1.0   

   ...  NewCases  NewDeaths  SeriousCritical  Tests1M  TotCases1M  \
0  ...       NaN        NaN              NaN      NaN         NaN   
1  ...       NaN        NaN              NaN      NaN         NaN   
2  ...       NaN        NaN              NaN      NaN         NaN   
3  ...       NaN        NaN              NaN      NaN         NaN   
4  ...       NaN        NaN              NaN      NaN         NaN   

   TotDeaths1M  TotalCases  TotalDeaths  TotalRecovered  TotalTests  
0          NaN         NaN          NaN             NaN         NaN  
1          NaN         NaN          NaN             NaN         NaN  
2          NaN         NaN          NaN             NaN         NaN  
3          NaN         NaN          NaN             NaN         NaN  
4          NaN         NaN          NaN             NaN         NaN  

[5 rows x 178 columns]

In [14]:
wm1

Date  ActiveCases  Deaths1M  NewCases  NewDeaths  SeriousCritical  \
0     2020-02-23          NaN       NaN  0.693147        NaN              NaN   
1     2020-02-25          NaN       NaN       NaN        NaN              NaN   
2     2020-02-26          NaN       NaN       NaN        NaN              NaN   
3     2020-02-27          NaN       NaN       NaN        NaN              NaN   
4     2020-02-28     0.693147       NaN       NaN        NaN              NaN   
...          ...          ...       ...       ...        ...              ...   
12691 2020-04-10     2.397895  0.182322       NaN        NaN              NaN   
12692 2020-04-11     2.484907  0.182322       NaN        NaN              NaN   
12693 2020-04-12     2.708050  0.182322       NaN        NaN              NaN   
12694 2020-04-13     2.708050  0.182322       NaN        NaN              NaN   
12695 2020-04-14     2.995732  0.182322  1.945910        NaN              NaN   

        Tests1M  TotCases1M  TotDeaths1M  TotalCases  TotalDeaths  \
0           NaN         NaN          NaN    0.693147          NaN   
1           NaN         NaN          NaN    0.693147          NaN   
2           NaN         NaN          NaN    0.693147          NaN   
3           NaN         NaN          NaN    0.693147          NaN   
4           NaN         NaN          NaN    0.693147          NaN   
...         ...         ...          ...         ...          ...   
12691  3.401197    0.641854          NaN    2.639057     1.386294   
12692  3.637586    0.641854          NaN    2.708050     1.386294   
12693  3.737670    0.693147          NaN    2.890372     1.386294   
12694  3.737670    0.693147          NaN    2.890372     1.386294   
12695  3.891820    1.098612          NaN    3.178054     1.386294   

       TotalRecovered  TotalTests Country_Region Province_State  
0                 NaN         NaN    Afghanistan            NaN  
1                 NaN         NaN    Afghanistan            NaN  
2                 NaN         NaN    Afghanistan            NaN  
3                 NaN         NaN    Afghanistan            NaN  
4                 NaN         NaN    Afghanistan            NaN  
...               ...         ...            ...            ...  
12691             NaN    6.084499       Zimbabwe            NaN  
12692             NaN    6.306275       Zimbabwe            NaN  
12693             NaN    6.405228       Zimbabwe            NaN  
12694             NaN    6.405228       Zimbabwe            NaN  
12695        0.693147    6.575076       Zimbabwe            NaN  

[12696 rows x 15 columns]

In [15]:
# feature names
names=["lag_confirmed_rate" + str(k+1) for k in range (size)]
for day in days_back_confimed:
    names+=["days_ago_confirmed_count_" + str(day) ]
for window in windows:        
    names+=["ma" + str(window) + "_rate_confirmed" + str(k+1) for k in range (size)]
    names+=["std" + str(window) + "_rate_confirmed" + str(k+1) for k in range (size)] 
    names+=["ewma" + str(window) + "_rate_confirmed" + str(k+1) for k in range (size)]         


names+=["lag_fatalities_rate" + str(k+1) for k in range (size)]
for day in days_back_fatalities:
    names+=["days_ago_fatalitiescount_" + str(day) ]    
for window in windows:        
    names+=["ma" + str(window) + "_rate_fatalities" + str(k+1) for k in range (size)]
    names+=["std" + str(window) + "_rate_fatalities" + str(k+1) for k in range (size)]  
    names+=["ewma" + str(window) + "_rate_fatalities" + str(k+1) for k in range (size)]        
names+=["confirmed_level"]
names+=["fatalities_level"]   
if not extra_stable_columns is None and len(extra_stable_columns)>0: 
    names+=[k for k in extra_stable_columns]  
    
if not group_names is None:  
     for gg in range (len(group_names)):
         names+=["lag_rate_group_"+ str(gg+1) + "_" + str(k+1) for k in range (size_group)]    
         for day in days_back_confimed_group:
            names+=["days_ago_grooupcount_" + str(gg+1) + "_" + str(day) ]             
         for window in windows_group:        
            names+=["ma_group_" + str(gg+1) + "_" + str(window) + "_rate_" + str(k+1) for k in range (size_group)]
            names+=["std_group_" + str(gg+1)+ "_" + str(window) + "_rate_" + str(k+1) for k in range (size_group)]  
            #names+=["ewma_group_" + str(gg+1) + "_" + str(window) + "_rate_" + str(k+1) for k in range (size)]      
 
names.extend(wmf)

print(names, len(names))

['lag_confirmed_rate1', 'lag_confirmed_rate2', 'lag_confirmed_rate3', 'lag_confirmed_rate4', 'lag_confirmed_rate5', 'lag_confirmed_rate6', 'lag_confirmed_rate7', 'lag_confirmed_rate8', 'lag_confirmed_rate9', 'lag_confirmed_rate10', 'days_ago_confirmed_count_1', 'days_ago_confirmed_count_5', 'days_ago_confirmed_count_10', 'days_ago_confirmed_count_20', 'days_ago_confirmed_count_50', 'days_ago_confirmed_count_100', 'days_ago_confirmed_count_250', 'days_ago_confirmed_count_500', 'days_ago_confirmed_count_1000', 'ma3_rate_confirmed1', 'ma3_rate_confirmed2', 'ma3_rate_confirmed3', 'ma3_rate_confirmed4', 'ma3_rate_confirmed5', 'ma3_rate_confirmed6', 'ma3_rate_confirmed7', 'ma3_rate_confirmed8', 'ma3_rate_confirmed9', 'ma3_rate_confirmed10', 'std3_rate_confirmed1', 'std3_rate_confirmed2', 'std3_rate_confirmed3', 'std3_rate_confirmed4', 'std3_rate_confirmed5', 'std3_rate_confirmed6', 'std3_rate_confirmed7', 'std3_rate_confirmed8', 'std3_rate_confirmed9', 'std3_rate_confirmed10', 'ewma3_rate_co

In [16]:
# save training data frame
fname = mname + '_train_frame.csv'
train_frame.to_csv(fname, index=False)
print(fname, train_frame.shape)

kaz0z_train_frame.csv (26292, 178)


In [17]:
train_frame.head()

Id Province_State Country_Region       Date  ConfirmedCases  Fatalities  \
0   1            NaN    Afghanistan 2020-01-22             0.0         0.0   
1   2            NaN    Afghanistan 2020-01-23             0.0         0.0   
2   3            NaN    Afghanistan 2020-01-24             0.0         0.0   
3   4            NaN    Afghanistan 2020-01-25             0.0         0.0   
4   5            NaN    Afghanistan 2020-01-26             0.0         0.0   

               key  rate_ConfirmedCases  rate_Fatalities  lag_confirmed_rate1  \
0  nan_Afghanistan                  1.0              1.0                  1.0   
1  nan_Afghanistan                  1.0              1.0                  1.0   
2  nan_Afghanistan                  1.0              1.0                  1.0   
3  nan_Afghanistan                  1.0              1.0                  1.0   
4  nan_Afghanistan                  1.0              1.0                  1.0   

   ...  NewCases  NewDeaths  SeriousCritical  Tests1M  TotCases1M  \
0  ...       NaN        NaN              NaN      NaN         NaN   
1  ...       NaN        NaN              NaN      NaN         NaN   
2  ...       NaN        NaN              NaN      NaN         NaN   
3  ...       NaN        NaN              NaN      NaN         NaN   
4  ...       NaN        NaN              NaN      NaN         NaN   

   TotDeaths1M  TotalCases  TotalDeaths  TotalRecovered  TotalTests  
0          NaN         NaN          NaN             NaN         NaN  
1          NaN         NaN          NaN             NaN         NaN  
2          NaN         NaN          NaN             NaN         NaN  
3          NaN         NaN          NaN             NaN         NaN  
4          NaN         NaN          NaN             NaN         NaN  

[5 rows x 178 columns]

In [18]:
import lightgbm as lgb
from sklearn.linear_model import Ridge
from sklearn.externals import joblib
import gc

# # from oscii
# SEED = 42
# params = {'num_leaves': 8,
#           'min_data_in_leaf': 5,  # 42,
#           'objective': 'regression',
#           'max_depth': 8,
#           'learning_rate': 0.02,
#           'n_estimators': 100,
#           'boosting': 'gbdt',
#           'bagging_freq': 5,  # 5
#           'bagging_fraction': 0.8,  # 0.5,
#           'feature_fraction': 0.8201,
#           'bagging_seed': SEED,
#           'reg_alpha': 1,  # 1.728910519108444,
#           'reg_lambda': 4.9847051755586085,
#           'random_state': SEED,
#           'metric': 'rmse',
#           # 'verbosity': 100,
#           'min_gain_to_split': 0.02,  # 0.01077313523861969,
#           'min_child_weight': 5,  # 19.428902804238373,
#           # 'num_threads': 6,
#           }

params = {'objective': 'rmse', 'metric': 'rmse', 'boosting': 'gbdt', 'learning_rate': 0.005,
          'drop_rate': 0.01,
          'skip_drop': 0.6, 'uniform_drop': True, 'verbose': -1, 'num_leaves': 30,
          'bagging_fraction': 0.9,
          'bagging_freq': 1, 'bagging_seed': 1412, 'feature_fraction': 0.8,
          'feature_fraction_seed': 1412,
          'min_data_in_leaf': 10, 'max_bin': 100, 'max_depth': 20, 'reg_alpha': 1,
          'lambda_l2': 10, 'num_threads': -1,
          'n_estimators': 500}

kwargs = {'verbose':False}

/home/sasrdw/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [19]:
horizon

30

In [20]:
train_frame.shape

(26292, 178)

In [21]:
# training loop
d = train_frame
mnam = ['confirmed','fatal']
ynam = ['confirmed_plus','fatalities_plus']
for h in range(horizon):
    hs = str(h)
    hs1 = str(h+1)
    q = d[ynam[0]+hs1] > 0
    # dupp = max_train_date - timedelta(h+1)
    # dlow = max_train_date - timedelta(h+1+ntrain)
    # q = (dlow < d.Date) & (d.Date <= dupp)
    xfull = d.loc[q, names].copy()
    wfull = np.log(d.loc[q,'ConfirmedCases'] + 2.) 
    for i,y in enumerate(ynam):
        yfull = d.loc[q,y+hs1].values
        # wfull = np.log(d.loc[q,'ConfirmedCases'] + 2.) if i==0 else np.log(d.loc[q,'Fatalities'] + 2.)
        # m = joblib.load(model_directory + 'model' + mnam[i] + hs)
        # params = m.params
        # params['num_threads'] = -1
        # params['n_estimators'] = 1000 if i==0 else 300
        # params['n_estimators'] = m.current_iteration()
        if (h<=0): print(params)
        for j in range(nbag):
            js = '_b' + str(j)
            params['bagging_seed'] = seed + j
            params['feature_fraction_seed'] = seed + j
            model = lgb.LGBMRegressor(**params)
            model.fit(xfull, yfull, sample_weight=wfull, **kwargs)
            fname = model_directory + mname + '_' + y[:1] + hs + js
            joblib.dump(model, fname)
            print('fit horizon', hs1, y, xfull.shape, fname)
gc.collect()

{'objective': 'rmse', 'metric': 'rmse', 'boosting': 'gbdt', 'learning_rate': 0.005, 'drop_rate': 0.01, 'skip_drop': 0.6, 'uniform_drop': True, 'verbose': -1, 'num_leaves': 30, 'bagging_fraction': 0.9, 'bagging_freq': 1, 'bagging_seed': 1412, 'feature_fraction': 0.8, 'feature_fraction_seed': 1412, 'min_data_in_leaf': 10, 'max_bin': 100, 'max_depth': 20, 'reg_alpha': 1, 'lambda_l2': 10, 'num_threads': -1, 'n_estimators': 500}
fit horizon 1 confirmed_plus (25979, 109) ./modelv4-dir/kaz0z_c0_b0
{'objective': 'rmse', 'metric': 'rmse', 'boosting': 'gbdt', 'learning_rate': 0.005, 'drop_rate': 0.01, 'skip_drop': 0.6, 'uniform_drop': True, 'verbose': -1, 'num_leaves': 30, 'bagging_fraction': 0.9, 'bagging_freq': 1, 'bagging_seed': 456, 'feature_fraction': 0.8, 'feature_fraction_seed': 456, 'min_data_in_leaf': 10, 'max_bin': 100, 'max_depth': 20, 'reg_alpha': 1, 'lambda_l2': 10, 'num_threads': -1, 'n_estimators': 500}
fit horizon 1 fatalities_plus (25979, 109) ./modelv4-dir/kaz0z_f0_b0
fit horiz

41

In [22]:
# prediction function using presaved models
def predict(xtest,input_name=None):
    # print (type(yt))
    # create array object to hold predictions 

    baggedpred = np.array([0.0 for d in range(0, xtest.shape[0])]) 
    for j in range(nbag):
        model = joblib.load(input_name + '_b' + str(j)) 
        preds = model.predict(xtest)               
        baggedpred += preds
        
    baggedpred /= nbag

    return baggedpred

In [23]:
# post-processing functions
def decay_4_first_10_then_1_f(array):
    arr=[1.0 for k in range(len(array))]
    for j in range(len(array)):
        if j<10:
            arr[j]=1. + (max(1,array[j])-1.)/4.
        else :
            arr[j]=1.
    return arr

def decay_16_first_10_then_1_f(array):
    arr=[1.0 for k in range(len(array))]
    for j in range(len(array)):
        if j<10:
            arr[j]=1. + (max(1,array[j])-1.)/16.
        else :
            arr[j]=1.
    return arr	
            
def decay_2_f(array):
    arr=[1.0 for k in range(len(array))]    
    for j in range(len(array)):
            arr[j]=1. + (max(1,array[j])-1.)/2.
    return arr 

def decay_4_f(array):
    arr=[1.0 for k in range(len(array))]    
    for j in range(len(array)):
            arr[j]=1. + (max(1,array[j])-1.)/4.
    return arr 	

def acceleratorx2_f(array):
    arr=[1.0 for k in range(len(array))]    
    for j in range(len(array)):
            arr[j]=1. + (max(1,array[j])-1.)*2.
    return arr 

def decay_1_5_f(array):
    arr=[1.0 for k in range(len(array))]    
    for j in range(len(array)):
            arr[j]=1. + (max(1,array[j])-1.)/1.5
    return arr            
         
         
def stay_same_f(array):
    arr=[1.0 for k in range(len(array))]      
    for j in range(len(array)):
        arr[j]=1.
    return arr   

def decay_2_last_12_linear_inter_f(array):
    arr=[1.0 for k in range(len(array))]
    for j in range(len(array)):
        arr[j]=1. + (max(1,array[j])-1.)/2.
    arr12= (max(1,arr[-12])-1.)/12. 

    for j in range(0, 12):
        arr[len(arr)-12 +j]= max(1, 1 + ( (arr12*12) - (j+1)*arr12 ))
    return arr

def decay_4_last_12_linear_inter_f(array):
    arr=[1.0 for k in range(len(array))]
    for j in range(len(array)):
        arr[j]=1. + (max(1,array[j])-1.)/4.
    arr12= (max(1,arr[-12])-1.)/12. 

    for j in range(0, 12):
        arr[len(arr)-12 +j]= max(1, 1 + ( (arr12*12) - (j+1)*arr12 ))
    return arr

def linear_last_12_f(array):
    arr=[1.0 for k in range(len(array))]
    for j in range(len(array)):
        arr[j]=max(1,array[j])
    arr12= (max(1,arr[-12])-1.)/12. 
    
    for j in range(0, 12):
        arr[len(arr)-12 +j]= max(1, 1 + ( (arr12*12) - (j+1)*arr12 ))
    return arr
   
decay_4_first_10_then_1 =[ "Heilongjiang_China","Liaoning_China","Shanghai_China"]#, "Hong Kong_China"

decay_4_first_10_then_1_fatality=[]

decay_16_first_10_then_1 =["Beijing_China","Fujian_China","Guangdong_China","Shandong_China","Sichuan_China","Zhejiang_China"]
decay_16_first_10_then_1_fatality=[]

decay_4=["nan_Bhutan","nan_Burundi","nan_Cabo Verde","Prince Edward Island_Canada",
"nan_Central African Republic","Inner Mongolia_China","nan_Maldives","Falkland Islands (Malvinas)_United Kingdom"]

decay_4_fatality=["nan_Congo (Kinshasa)"]

decay_2 =["nan_Congo (Kinshasa)","Faroe Islands_Denmark","nan_Eritrea","French Guiana_France","nan_Korea, South","nan_MS Zaandam"]
decay_2_fatality=[]

stay_same=["nan_Diamond Princess","nan_Timor-Leste"]
  
stay_same_fatality=["Beijing_China","Fujian_China","Guangdong_China","Shandong_China",
"Sichuan_China","Zhejiang_China", "Heilongjiang_China","Liaoning_China","Shanghai_China"]#

normal=[]
normal_fatality=["nan_Korea, South","New York_US"]

decay_4_last_12_linear_inter =[ "Greenland_Denmark","nan_Dominica","nan_Equatorial Guinea","nan_Eswatini","New Caledonia_France",
"Saint Barthelemy_France","St Martin_France","nan_Gambia","nan_Grenada","nan_Holy See","nan_Mauritania","nan_Namibia","nan_Nicaragua"
,"nan_Papua New Guinea","nan_Saint Lucia","nan_Saint Vincent and the Grenadines","nan_Seychelles","nan_Sierra Leone","nan_Somalia","nan_Suriname",
"Anguilla_United Kingdom","British Virgin Islands_United Kingdom","Montserrat_United Kingdom","Turks and Caicos Islands_United Kingdom",
"nan_Zimbabwe", "Hong Kong_China","Curacao_Netherlands",
"Saint Pierre and Miquelon_France","nan_South Sudan","nan_Western Sahara",
"nan_Malawi","Bonaire, Sint Eustatius and Saba_Netherlands","nan_Sao Tome and Principe"
]

decay_4_last_12_linear_inter_fatality=[]

decay_2_last_12_linear_inter =[ "nan_Chad",
"nan_Congo (Brazzaville)","nan_Fiji","French Polynesia_France","nan_Gabon",
"nan_Guyana","nan_Laos","nan_Nepal","Sint Maarten_Netherlands",
"nan_Saint Kitts and Nevis","nan_Sudan","nan_Syria","nan_Tanzania",
"Bermuda_United Kingdom","Cayman Islands_United Kingdom","nan_Zambia","Northwest Territories_Canada","Yukon_Canada"
,"nan_Mongolia","nan_Uganda"]
decay_2_last_12_linear_inter_fatality=[]

acceleratorx2=[]
acceleratorx2_fatality=[]


warm_st=['nan_Angola','nan_Antigua and Barbuda','Northern Territory_Australia','nan_Bahamas',
'nan_Bangladesh','nan_Belize','nan_Benin','nan_Botswana','nan_Burundi','nan_Cabo Verde','nan_Cameroon',
'nan_Central African Republic','nan_Chad','Hong Kong_China',"nan_Cote d'Ivoire",'nan_Cuba','Greenland_Denmark',
'nan_Dominica','nan_Equatorial Guinea','nan_Eritrea','nan_Eswatini','nan_Fiji','French Polyneta_France','New Caledonia_France',
'Saint Barthelemy_France','St Martin_France','nan_Gabon','nan_Gambia','nan_Grenada','nan_Guyana','nan_Haiti','nan_Holy See',
'nan_Honduras','nan_Ireland','nan_Korea, South','nan_Laos','nan_Liberia','nan_Libya','nan_Maldives','nan_Mali',
'nan_Mauritania','nan_Mauritius','nan_Mongolia','nan_Mozambique','nan_Namibia','nan_Nepal','Aruba_Netherlands',
'nan_Nicaragua','nan_Niger','nan_Papua New Guinea','nan_Saint Kitts and Nevis','nan_Saint Lucia',
'nan_Saint Vincent and the Grenadines','nan_Seychelles','nan_Sierra Leone','nan_Somalia',
'nan_Spain','nan_Sudan','nan_Suriname','nan_Syria','nan_Tanzania','nan_Togo','nan_Uganda','Anguilla_United Kingdom',
'Bermuda_United Kingdom','British Virgin Islands_United Kingdom','Channel Islands_United Kingdom',
'Gibraltar_United Kingdom','Isle of Man_United Kingdom','Montserrat_United Kingdom','nan_United Kingdom',
'Turks and Caicos Islands_United Kingdom','nan_Uzbekistan','nan_Zimbabwe',
'Saint Pierre and Miquelon_France','nan_South Sudan','nan_Western Sahara',
'nan_Malawi','Bonaire, Sint Eustatius and Saba_Netherlands','nan_Sao Tome and Principe',
'Falkland Islands (Malvinas)_United Kingdom'
]


decay_1_5 =["nan_Angola" ,"nan_Antigua and Barbuda","Montana_US","Nebraska_US","nan_Bangladesh","Illinois_US"
,"Northern Territory_Australia","nan_Bahamas","nan_Bahrain","nan_Barbados" ,"nan_Belize","nan_Benin",
	"nan_Botswana","nan_Brunei","Manitoba_Canada","New Brunswick_Canada","Saskatchewan_Canada",
	"nan_Cote d'Ivoire","nan_France","nan_Guinea-Bissau","nan_Haiti","nan_Italy","nan_Libya","nan_Malta","nan_Mauritius",
	"Aruba_Netherlands","nan_Niger","nan_Spain","nan_Togo","Guam_US","Iowa_US","Idaho_US","Connecticut_US","California_US","New York_US","Virgin Islands_US",
	"Channel Islands_United Kingdom","Gibraltar_United Kingdom","Isle of Man_United Kingdom","nan_United Kingdom",'nan_Burma']

decay_1_5_fatality=["nan_Cameroon","nan_Mali","nan_Cuba","Delaware_US","District of Columbia_US",
"Kansas_US","Louisiana_US","Michigan_US","New Mexico_US","Ohio_US","Oklahoma_US","Pennsylvania_US","Puerto Rico_US","Rhode Island_US",
"South Dakota_US" ,"Tennessee_US","Texas_US","Vermont_US","Virginia_US","West Virginia_US","nan_Uzbekistan"]

linear_last_12=["nan_Honduras","nan_Ireland","Colorado_US","nan_Liberia","nan_Mozambique"]
linear_last_12_fatality=[]

In [24]:
train_frame.shape

(26292, 178)

In [25]:
train_frame

Id Province_State Country_Region       Date  ConfirmedCases  \
0          1            NaN    Afghanistan 2020-01-22             0.0   
1          2            NaN    Afghanistan 2020-01-23             0.0   
2          3            NaN    Afghanistan 2020-01-24             0.0   
3          4            NaN    Afghanistan 2020-01-25             0.0   
4          5            NaN    Afghanistan 2020-01-26             0.0   
...      ...            ...            ...        ...             ...   
26287  35648            NaN       Zimbabwe 2020-04-10            13.0   
26288  35649            NaN       Zimbabwe 2020-04-11            14.0   
26289  35650            NaN       Zimbabwe 2020-04-12            14.0   
26290  35651            NaN       Zimbabwe 2020-04-13            17.0   
26291  35652            NaN       Zimbabwe 2020-04-14            17.0   

       Fatalities              key  rate_ConfirmedCases  rate_Fatalities  \
0             0.0  nan_Afghanistan             1.000000              1.0   
1             0.0  nan_Afghanistan             1.000000              1.0   
2             0.0  nan_Afghanistan             1.000000              1.0   
3             0.0  nan_Afghanistan             1.000000              1.0   
4             0.0  nan_Afghanistan             1.000000              1.0   
...           ...              ...                  ...              ...   
26287         3.0     nan_Zimbabwe             1.181818              1.0   
26288         3.0     nan_Zimbabwe             1.076923              1.0   
26289         3.0     nan_Zimbabwe             1.000000              1.0   
26290         3.0     nan_Zimbabwe             1.214286              1.0   
26291         3.0     nan_Zimbabwe             1.000000              1.0   

       lag_confirmed_rate1  ...  NewCases  NewDeaths  SeriousCritical  \
0                 1.000000  ...       NaN        NaN              NaN   
1                 1.000000  ...       NaN        NaN              NaN   
2                 1.000000  ...       NaN        NaN              NaN   
3                 1.000000  ...       NaN        NaN              NaN   
4                 1.000000  ...       NaN        NaN              NaN   
...                    ...  ...       ...        ...              ...   
26287             1.181818  ...       NaN        NaN              NaN   
26288             1.076923  ...       NaN        NaN              NaN   
26289             1.000000  ...       NaN        NaN              NaN   
26290             1.214286  ...       NaN        NaN              NaN   
26291             1.000000  ...   1.94591        NaN              NaN   

        Tests1M  TotCases1M  TotDeaths1M  TotalCases  TotalDeaths  \
0           NaN         NaN          NaN         NaN          NaN   
1           NaN         NaN          NaN         NaN          NaN   
2           NaN         NaN          NaN         NaN          NaN   
3           NaN         NaN          NaN         NaN          NaN   
4           NaN         NaN          NaN         NaN          NaN   
...         ...         ...          ...         ...          ...   
26287  3.401197    0.641854          NaN    2.639057     1.386294   
26288  3.637586    0.641854          NaN    2.708050     1.386294   
26289  3.737670    0.693147          NaN    2.890372     1.386294   
26290  3.737670    0.693147          NaN    2.890372     1.386294   
26291  3.891820    1.098612          NaN    3.178054     1.386294   

       TotalRecovered  TotalTests  
0                 NaN         NaN  
1                 NaN         NaN  
2                 NaN         NaN  
3                 NaN         NaN  
4                 NaN         NaN  
...               ...         ...  
26287             NaN    6.084499  
26288             NaN    6.306275  
26289             NaN    6.405228  
26290             NaN    6.405228  
26291        0.693147    6.575076  

[26292 rows x 178 columns]

In [26]:
len(names)

109

In [27]:
# build lists for prediction
tr_frame = train_frame

features_train = tr_frame[names].values   

standard_confirmed_train = tr_frame["ConfirmedCases"].values
standard_fatalities_train = tr_frame["Fatalities"].values
current_confirmed_train = tr_frame["ConfirmedCases"].values

features_cv = []
name_cv = []
standard_confirmed_cv = []
standard_fatalities_cv = []
names_ = tr_frame["key"].values
training_horizon = int(features_train.shape[0]/len(unique_keys)) 
print("training horizon = ",training_horizon)

# use final row for each location as its features
for dd in range(training_horizon-1, features_train.shape[0], training_horizon):
    features_cv.append(features_train[dd])
    name_cv.append(names_[dd])
    standard_confirmed_cv.append(standard_confirmed_train[dd])
    standard_fatalities_cv.append(standard_fatalities_train[dd])
    print(dd, name_cv[-1], standard_confirmed_cv[-1], standard_fatalities_cv[-1])

training horizon =  84
83 nan_Afghanistan 714.0 23.0
167 nan_Albania 475.0 24.0
251 nan_Algeria 2070.0 326.0
335 nan_Andorra 659.0 31.0
419 nan_Angola 19.0 2.0
503 nan_Antigua and Barbuda 23.0 2.0
587 nan_Argentina 2277.0 102.0
671 nan_Armenia 1067.0 16.0
755 Australian Capital Territory_Australia 103.0 2.0
839 New South Wales_Australia 2870.0 25.0
923 Northern Territory_Australia 28.0 0.0
1007 Queensland_Australia 998.0 5.0
1091 South Australia_Australia 433.0 4.0
1175 Tasmania_Australia 165.0 6.0
1259 Victoria_Australia 1291.0 14.0
1343 Western Australia_Australia 527.0 6.0
1427 nan_Austria 14226.0 384.0
1511 nan_Azerbaijan 1197.0 13.0
1595 nan_Bahamas 49.0 8.0
1679 nan_Bahrain 1528.0 7.0
1763 nan_Bangladesh 1012.0 46.0
1847 nan_Barbados 72.0 4.0
1931 nan_Belarus 3281.0 33.0
2015 nan_Belgium 31119.0 4157.0
2099 nan_Belize 18.0 2.0
2183 nan_Benin 35.0 1.0
2267 nan_Bhutan 5.0 0.0
2351 nan_Bolivia 354.0 28.0
2435 nan_Bosnia and Herzegovina 1083.0 40.0
2519 nan_Botswana 13.0 1.0
2603 nan

In [28]:
horizon

30

In [29]:
# set up for prediction
features_cv = np.array(features_cv)
preds_confirmed_cv = np.zeros((features_cv.shape[0], horizon))
preds_confirmed_standard_cv = np.zeros((features_cv.shape[0], horizon))

preds_fatalities_cv = np.zeros((features_cv.shape[0], horizon))
preds_fatalities_standard_cv = np.zeros((features_cv.shape[0], horizon))

overal_rmsle_metric_confirmed = 0.0

# predict confirmed cases
for j in range (preds_confirmed_cv.shape[1]):

    this_features_cv = features_cv                          

    preds = predict(features_cv, input_name=model_directory + mname + "_c"+ str(j))
    preds_confirmed_cv[:,j] = preds
    print ("predicting confirmed, horizon %d, original cv %d and after %d "% \
           (j+1,this_features_cv.shape[0],this_features_cv.shape[0])) 

predicting confirmed, horizon 1, original cv 313 and after 313 
predicting confirmed, horizon 2, original cv 313 and after 313 
predicting confirmed, horizon 3, original cv 313 and after 313 
predicting confirmed, horizon 4, original cv 313 and after 313 
predicting confirmed, horizon 5, original cv 313 and after 313 
predicting confirmed, horizon 6, original cv 313 and after 313 
predicting confirmed, horizon 7, original cv 313 and after 313 
predicting confirmed, horizon 8, original cv 313 and after 313 
predicting confirmed, horizon 9, original cv 313 and after 313 
predicting confirmed, horizon 10, original cv 313 and after 313 
predicting confirmed, horizon 11, original cv 313 and after 313 
predicting confirmed, horizon 12, original cv 313 and after 313 
predicting confirmed, horizon 13, original cv 313 and after 313 
predicting confirmed, horizon 14, original cv 313 and after 313 
predicting confirmed, horizon 15, original cv 313 and after 313 
predicting confirmed, horizon 16, 

In [30]:
# post-processsing overrides for confirmed
predictions=[] 
for ii in range (preds_confirmed_cv.shape[0]):
    current_prediction = standard_confirmed_cv[ii]
    if current_prediction==0 :
        current_prediction=0.1
        
    this_preds = preds_confirmed_cv[ii].tolist()
    name = name_cv[ii]
    reserve = this_preds[0]
    #overrides

    if name in normal:
    # if True:
        this_preds=this_preds
    
    elif name in decay_4_first_10_then_1:
        this_preds=decay_4_first_10_then_1_f(this_preds)

    elif name in decay_16_first_10_then_1:
        this_preds=decay_16_first_10_then_1_f(this_preds)

    elif name in decay_4_last_12_linear_inter:
        this_preds=decay_4_last_12_linear_inter_f(this_preds)       
          
    elif name in decay_4:
        this_preds=decay_4_f(this_preds)

    elif name in decay_2:
        this_preds=decay_2_f(this_preds)
        
    elif name in decay_2_last_12_linear_inter:
        this_preds=decay_2_last_12_linear_inter_f(this_preds)
        
    elif name in decay_1_5:
        this_preds=decay_1_5_f(this_preds)        
        
    elif name in linear_last_12:
        this_preds=linear_last_12_f(this_preds)
        
    elif name in acceleratorx2:
        this_preds=acceleratorx2_f(this_preds)         

    elif name in stay_same or  "China" in name:
        this_preds=stay_same_f(this_preds)      

    if name in warm_st:
        this_preds[0]=reserve
        
    for j in range (preds_confirmed_cv.shape[1]):
        current_prediction *= max(1,this_preds[j])
        preds_confirmed_standard_cv[ii][j] = current_prediction

In [31]:
# predict fatalities
for j in range (preds_confirmed_cv.shape[1]):
    this_features_cv = features_cv             
    preds=predict(features_cv, input_name=model_directory + mname + "_f"+ str(j))
    preds_fatalities_cv[:,j]=preds
    print ("predicting fatalities, horizon %d, original cv %d and after %d "% \
           (j+1,this_features_cv.shape[0],this_features_cv.shape[0])) 

predicting fatalities, horizon 1, original cv 313 and after 313 
predicting fatalities, horizon 2, original cv 313 and after 313 
predicting fatalities, horizon 3, original cv 313 and after 313 
predicting fatalities, horizon 4, original cv 313 and after 313 
predicting fatalities, horizon 5, original cv 313 and after 313 
predicting fatalities, horizon 6, original cv 313 and after 313 
predicting fatalities, horizon 7, original cv 313 and after 313 
predicting fatalities, horizon 8, original cv 313 and after 313 
predicting fatalities, horizon 9, original cv 313 and after 313 
predicting fatalities, horizon 10, original cv 313 and after 313 
predicting fatalities, horizon 11, original cv 313 and after 313 
predicting fatalities, horizon 12, original cv 313 and after 313 
predicting fatalities, horizon 13, original cv 313 and after 313 
predicting fatalities, horizon 14, original cv 313 and after 313 
predicting fatalities, horizon 15, original cv 313 and after 313 
predicting fataliti

In [32]:
# post-processing overrides for fatalities
predictions=[]
for ii in range (preds_fatalities_cv.shape[0]):
    current_prediction=standard_fatalities_cv[ii]
    if current_prediction==0 and standard_confirmed_cv[ii]>400:
        current_prediction=0.1
        
    this_preds=preds_fatalities_cv[ii].tolist()
    name=name_cv[ii]
    reserve=this_preds[0]
    #overrides
   
    ####fatality special
    if name in normal_fatality:
    # if True:
        this_preds=this_preds

    elif name in decay_4_first_10_then_1_fatality:
        this_preds=decay_4_first_10_then_1_f(this_preds) 

    elif name in decay_16_first_10_then_1_fatality:
        this_preds=decay_16_first_10_then_1_f(this_preds)
        
    elif name in decay_4_last_12_linear_inter_fatality:
        this_preds=decay_4_last_12_linear_inter_f(this_preds)           

    elif name in decay_4_fatality:
        this_preds=decay_4_f(this_preds)

    elif name in decay_2_fatality:
        this_preds=decay_2_f(this_preds)        

    elif name in decay_2_last_12_linear_inter_fatality:
        this_preds=decay_2_last_12_linear_inter_f(this_preds)
        
    elif name in decay_1_5_fatality:
        this_preds=decay_1_5_f(this_preds)   
 
    elif name in linear_last_12_fatality:
        this_preds=linear_last_12_f(this_preds) 
         
    elif name in acceleratorx2_fatality:
        this_preds=acceleratorx2_f(this_preds)
        
    elif name in stay_same_fatality:     
        this_preds=stay_same_f(this_preds) 
        
     
    ####general   
    elif name in normal:
        this_preds=this_preds  

    elif name in decay_4_first_10_then_1:
        this_preds=decay_4_first_10_then_1_f(this_preds)

    elif name in decay_16_first_10_then_1:
        this_preds=decay_16_first_10_then_1_f(this_preds)
        
    elif name in decay_4_last_12_linear_inter:
        this_preds=decay_4_last_12_linear_inter_f(this_preds)              

    elif name in decay_4:
        this_preds=decay_4_f(this_preds)        

    elif name in decay_2:
        this_preds=decay_2_f(this_preds)
        
    elif name in decay_2_last_12_linear_inter:
        this_preds=decay_2_last_12_linear_inter_f(this_preds)
        
    elif name in decay_1_5:
        this_preds=decay_1_5_f(this_preds)        
        
    elif name in linear_last_12:
        this_preds=linear_last_12_f(this_preds) 
        
    elif name in acceleratorx2:
        this_preds=acceleratorx2_f(this_preds)                 
        
    elif name in stay_same or  "China" in name:
        this_preds=stay_same_f(this_preds)         
       
    if name in warm_st:
        this_preds[0]=reserve    
    
    for j in range (preds_fatalities_cv.shape[1]):
                if current_prediction==0 and  (preds_confirmed_standard_cv[ii][j]>400 or "Malta" in name or "Somalia" in name):
                    current_prediction=1.
                if j==0 and "nan_Antigua and Barbuda" in name:
                    current_prediction=2.
                if j==0 and 'nan_Burma' in name:
                    current_prediction=3.                    
                current_prediction*=max(1,this_preds[j])
                preds_fatalities_standard_cv[ii][j]=current_prediction

In [33]:
# build keys
key_to_confirmed_rate={}
key_to_fatality_rate={}
key_to_confirmed={}
key_to_fatality={}
print(len(features_cv), len(name_cv),len(standard_confirmed_cv),len(standard_fatalities_cv)) 
print(preds_confirmed_cv.shape,preds_confirmed_standard_cv.shape,preds_fatalities_cv.shape,preds_fatalities_standard_cv.shape) 

for j in range (len(name_cv)):
    
    key_to_confirmed_rate[name_cv[j]]=preds_confirmed_cv[j,:].tolist()
    #print(key_to_confirmed_rate[name_cv[j]])
    key_to_fatality_rate[name_cv[j]]=preds_fatalities_cv[j,:].tolist()
    key_to_confirmed[name_cv[j]]  =preds_confirmed_standard_cv[j,:].tolist()  
    key_to_fatality[name_cv[j]]=preds_fatalities_standard_cv[j,:].tolist()  

313 313 313 313
(313, 30) (313, 30) (313, 30) (313, 30)


In [34]:
# set up data for submission
train_new=train[["Date","ConfirmedCases","Fatalities","key","rate_ConfirmedCases","rate_Fatalities"]]

test_new = pd.merge(test,train_new, how="left", left_on=["key","Date"], right_on=["key","Date"] ).reset_index(drop=True)
test_new

ForecastId Province_State Country_Region       Date              key  \
0               1            NaN    Afghanistan 2020-04-02  nan_Afghanistan   
1               2            NaN    Afghanistan 2020-04-03  nan_Afghanistan   
2               3            NaN    Afghanistan 2020-04-04  nan_Afghanistan   
3               4            NaN    Afghanistan 2020-04-05  nan_Afghanistan   
4               5            NaN    Afghanistan 2020-04-06  nan_Afghanistan   
...           ...            ...            ...        ...              ...   
13454       13455            NaN       Zimbabwe 2020-05-10     nan_Zimbabwe   
13455       13456            NaN       Zimbabwe 2020-05-11     nan_Zimbabwe   
13456       13457            NaN       Zimbabwe 2020-05-12     nan_Zimbabwe   
13457       13458            NaN       Zimbabwe 2020-05-13     nan_Zimbabwe   
13458       13459            NaN       Zimbabwe 2020-05-14     nan_Zimbabwe   

       ConfirmedCases  Fatalities  rate_ConfirmedCases  rate_Fatalities  
0               273.0         6.0             1.151899         1.500000  
1               281.0         6.0             1.029304         1.000000  
2               299.0         7.0             1.064057         1.166667  
3               349.0         7.0             1.167224         1.000000  
4               367.0        11.0             1.051576         1.571429  
...               ...         ...                  ...              ...  
13454             NaN         NaN                  NaN              NaN  
13455             NaN         NaN                  NaN              NaN  
13456             NaN         NaN                  NaN              NaN  
13457             NaN         NaN                  NaN              NaN  
13458             NaN         NaN                  NaN              NaN  

[13459 rows x 9 columns]

In [35]:
# fill in values for submission
def fillin_columns(frame,key_column, original_name, training_horizon, test_horizon, unique_values, key_to_values):
    keys = frame[key_column].values
    original_values = frame[original_name].values.tolist()
    print(len(keys), len(original_values), training_horizon ,test_horizon,len(key_to_values))
    
    for j in range(unique_values):
        current_index=(j * (training_horizon +test_horizon )) +training_horizon 
        current_key=keys[current_index]
        values=key_to_values[current_key]
        co = 0
        for g in range(current_index, current_index + test_horizon):
            original_values[g] = values[co]
            co += 1
    
    frame[original_name]=original_values
 

all_days=int(test_new.shape[0]/len(unique_keys))

tr_horizon=all_days-horizon
print(all_days,tr_horizon, horizon )

fillin_columns(test_new,"key", 'ConfirmedCases', tr_horizon, horizon, len(unique_keys), key_to_confirmed)    
fillin_columns(test_new,"key", 'Fatalities', tr_horizon, horizon, len(unique_keys), key_to_fatality) 

43 13 30
13459 13459 13 30 313
13459 13459 13 30 313


In [36]:
submission=test_new[["ForecastId","ConfirmedCases","Fatalities"]]

fname = 'sub/' + mname + '.csv'
submission.to_csv(fname, index=False)
print(fname, submission.shape)

sub/kaz0z.csv (13459, 3)


In [37]:
# m.__dict__

In [38]:
# m.model_to_string()